In [1]:
import numpy as np
import pandas as pd
import glob
import os
import pdb
os.environ["CUDA_VISIBLE_DEVICES"]='0'
from scipy import signal
from sklearn import preprocessing
import matplotlib.pyplot as plt


### 预处理之后再分类保存

In [3]:
people=14
data_path = r'/home/user/zhangshuailei/data/braindecode/csvFile/'+str(people) # 原始数据根目录
resultFolder = '/home/user/zhangshuailei/data/braindecode/csv_Siamese/'+str(people)+'_preprocess'
for i in range(4):
    csvFolderPath = resultFolder + '/' + str(i)
    folderExist = os.path.exists(csvFolderPath)
    if not folderExist:
        os.makedirs(csvFolderPath) # 创建文件夹
        
        
channel_pick = ['Fp1', 'Fp2', 'Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2',
                'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2',
                'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2', 'AF7',
                'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'C5',
                'C1', 'C2', 'C6', 'CP3', 'CPz', 'CP4', 'P5', 'P1', 'P2', 'P6', 'PO5',
                'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'FT9',
                'FT10', 'TPP9h', 'TPP10h', 'PO9', 'PO10', 'P9', 'P10', 'AFF1', 'AFz',
                'AFF2', 'FFC5h', 'FFC3h', 'FFC4h', 'FFC6h', 'FCC5h', 'FCC3h', 'FCC4h',
                'FCC6h', 'CCP5h', 'CCP3h', 'CCP4h', 'CCP6h', 'CPP5h', 'CPP3h', 'CPP4h',
                'CPP6h', 'PPO1', 'PPO2', 'I1', 'Iz', 'I2', 'AFp3h', 'AFp4h', 'AFF5h',
                'AFF6h', 'FFT7h', 'FFC1h', 'FFC2h', 'FFT8h', 'FTT9h', 'FTT7h', 'FCC1h',
                'FCC2h', 'FTT8h', 'FTT10h', 'TTP7h', 'CCP1h', 'CCP2h', 'TTP8h', 'TPP7h',
                'CPP1h', 'CPP2h', 'TPP8h', 'PPO9h', 'PPO5h', 'PPO6h', 'PPO10h', 'POO9h',
                'POO3h', 'POO4h', 'POO10h', 'OI1h', 'OI2h']


firOrderNum = 500 # 滤波器阶数
freq = 250 # 信号采样率
channel_num = len(channel_pick) # 通道数
time_num = 1000 # 时间点数

b = signal.firwin(firOrderNum,[4,30],nyq=freq/2,pass_zero=False)

data_list = glob.glob(os.path.join(data_path, '*.csv'))
for csvFile in data_list:
    sample = pd.read_csv(csvFile)#for循环里面还可以不是整数，循环的标记i也可以不是整数，可以是一个字符串
    sample_class = int(sample['STI 014'][250]) - 1#数据的第‘STI 014’通道是事件的通道，由于之前多取了500个点，本来这个事件标记应该在第一个的，但是由于2倍降采样，所以就是第251个
    #但是要考虑的是我的事件标记会不会在降采样的时候被抽掉了。这里要说一下，event[0:1000:2]表示的是第一个数就被留下来了，也就是奇数留下来。所有的抽取都是第一个数会留下来。
    #还有，为什么标签值减了1？因为神经网络的标签只能从0开始，这里的标签是1234，所以减去1
    sample_pick = pd.DataFrame(sample,columns=channel_pick) # 选取通道
    sample_pick = np.array(sample_pick).T # 转置 signal函数在处理数据时是沿着一个固定的维度，一般axis=-1，表示沿着最后一个维度，这里数据有两维，行是第一维，列是第二维，因此是沿着列来处理
    #打开数据保存的csv文件我们可以发现
#     pdb.set_trace()
    sample_detrend = signal.detrend(sample_pick) # detrend
    sample_filter = signal.lfilter(b,[1,],sample_detrend)[:,firOrderNum:] # FIR滤波
    sample_zscore = preprocessing.scale(sample_filter, axis=1) # z-score
    data = {}
    for i in range(128):
        data[channel_pick[i]] = sample_zscore[i,:].tolist()
    data = pd.DataFrame(data)
    data.to_csv(resultFolder + r'/' + str(sample_class) + '/' + csvFile.split('/')[-1])